In [32]:
%run ../Tarea_1./Tarea1_ej1.ipynb

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   _id_              50000 non-null  int64         
 1   _id2_             50000 non-null  int64         
 2   comp_mixto1       50000 non-null  float64       
 3   inversion         50000 non-null  float64       
 4   fecha_inicio      50000 non-null  datetime64[ns]
 5   comp_interno1     50000 non-null  float64       
 6   comp_interno2     50000 non-null  object        
 7   comp_interno3     50000 non-null  float64       
 8   comp_interno4     50000 non-null  float64       
 9   categoria1        50000 non-null  float64       
 10  categoria2        50000 non-null  float64       
 11  categoria3        50000 non-null  float64       
 12  comp_externo1     47831 non-null  float64       
 13  comp_externo2     44410 non-null  float64       
 14  comp_externo3     4999

In [29]:
final.head()

,_id_,_id2_,comp_mixto1,inversion,fecha_inicio,comp_interno1,comp_interno2,comp_interno3,comp_interno4,categoria1,categoria2,categoria3,comp_externo1,comp_externo2,comp_externo3,comp_externo4,mes_informacion,Tiempo_aparicion,comportamiento_j
0,78213,1798899,0.33522,0.000000,2020-03-25,0.000000,b'0',11.0,58.0,2.0,9.0,156.0,260.869565,25.854639,56.0,-230.0,2021-05-01,13.0,1.0
1,76457,1758511,0.33334,0.041039,2020-02-13,0.000000,b'1',28.0,45.0,1.0,4.0,173.0,40.579710,2.872738,62.0,-230.0,2021-05-01,15.0,1.0
2,40666,935318,0.29186,0.000000,2018-01-24,0.000000,b'1',25.0,45.0,1.0,9.0,170.0,0.000000,0.000000,119.0,-230.0,2019-05-01,15.0,1.0
3,8329,191567,0.29112,0.000000,2016-10-07,0.000000,b'1',14.0,59.0,3.0,3.0,159.0,349.275362,34.616490,94.0,-230.0,2021-05-01,44.0,0.0
4,68389,1572947,0.27318,77.422135,2019-07-30,2026.342452,b'1',1.0,67.0,2.0,9.0,146.0,276.811594,27.434645,125.0,-230.0,2021-05-01,20.0,1.0
